 # Ejercicio 3
 ## Realizar pruebas unitarias para asegurar el correcto funcionamiento de cada módulo.


In [ ]:
import pandas as pd
import unittest

url = "https://demodata.grapecity.com/contoso/api/v1/employees"
json_name = "employees.json"

class Test(unittest.TestCase):
    def readApi(self):
        # Test para ver si lee de la api
        df = self.dowload_data(url,json_name)
        self.assertFalse(df.empty)


    def filtros(self):
        #Test para ver si aplica filtros
        df = self.get_current_employees("")
        self.assertTrue(df['isCurrent'].all())
        
        df = self.get_married_employees("")
        self.assertTrue((df['Married'] == True).all())

        df = self.get_last_name_start_with("", "S")
        self.assertTrue(df['LastName'].str.startswith("S").all())

    def exportar(self):
        #Test para ver si exporta a csv
        nombre_archivo = "employees.csv"
        try:
            df = self.exportar("https://demodata.grapecity.com/contoso/api/v1/employees", nombre_archivo)
            self.assertTrue(nombre_archivo.endswith(".csv"))
            df = pd.read_csv(nombre_archivo)
            self.assertIsInstance(df, pd.DataFrame)
        except Exception as e:
            self.fail(f"No se pudo leer el archivo CSV: {e}")


if __name__ == '__main__':
    unittest.main()